# seedbleniding

exp40,exp49 のseed42,614


In [2]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

import pickle

In [3]:
train_42 = pd.read_csv("../output/exp21_Deeptables/exp21_deeptab_oof_07875390441655534.csv")
test_42 = pd.read_csv("../output/exp21_Deeptables/exp21_deeptab_test_07875390441655534.csv")

train_613 = pd.read_csv("../output/exp21_Deeptables/exp21_deeptab_oof_seed613.csv")
test_613 = pd.read_csv("../output/exp21_Deeptables/exp21_deeptab_test_seed613.csv")



In [4]:
train_42.head()

,customer_ID,target,exp21_deeptab_oof
0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.832788
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.020678
2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.999293
3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.001440
4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001569


In [5]:
train_613.head()

,customer_ID,target,exp21_deeptab_oof
0,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0.000493
1,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.001907
2,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,0,0.006730
3,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1,0.879497
4,000333075fb8ec6d504539852eeeb762643562e701ac79...,1,0.001352


In [6]:
oof = train_42.merge(train_613,on = "customer_ID")
sub = test_42.merge(test_613,on = "customer_ID")


In [7]:
sub.head()

,customer_ID,prediction_x,prediction_y
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.030432,0.027770
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000677,0.000672
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.026998,0.023484
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.346675,0.359638
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.873692,0.848255


In [8]:
oof.head()

,customer_ID,target_x,exp21_deeptab_oof_x,target_y,exp21_deeptab_oof_y
0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.832788,1,0.811039
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.020678,0,0.014285
2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.999293,1,0.999621
3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.001440,0,0.001023
4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001569,0,0.000494


In [9]:
oof["exp21_42+613_oof"] = (oof["exp21_deeptab_oof_x"] + oof["exp21_deeptab_oof_y"])/2
sub["prediction"] = (sub["prediction_x"] + sub["prediction_y"])/2 

In [10]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


score = amex_metric(oof["target_x"], oof["exp21_42+613_oof"])
print("CV : ",score)

CV :  0.7895385189225891


In [11]:
oof[["customer_ID","exp21_42+613_oof"]].to_csv(f"exp21_oof_deeptab_{score}_seedblend.csv",index = False)
sub[["customer_ID","prediction"]].to_csv(f"exp21_sub_deeptab_{score}_seedblend.csv",index = False)


In [13]:

amex_metric(oof["target_x"], oof["exp21_deeptab_oof_x"])

0.7873025333861761